# Determining the Fair Spread of a CDS Tranche

An analysis of pricing a CDS Index using its intrinsic value

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.funding import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  19 Nov 2020 at 22:37 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



In [15]:
valuationDate = FinDate(2, 8, 2007)
settlementDate = valuationDate.addWeekDays(1)

## Build Ibor Curve

In [16]:
dcType = FinDayCountTypes.THIRTY_E_360_ISDA
depos = []
depo = FinIborDeposit(valuationDate, "1D", 0.0500, dcType); depos.append(depo)
depo = FinIborDeposit(settlementDate, "1D", 0.0500, dcType); depos.append(depo)

fixedFreq = FinFrequencyTypes.SEMI_ANNUAL
swapType = FinSwapTypes.PAY
swap1 = FinIborSwap(settlementDate,"1Y",swapType,0.0502,fixedFreq,dcType)
swap2 = FinIborSwap(settlementDate,"2Y",swapType,0.0502,fixedFreq,dcType)
swap3 = FinIborSwap(settlementDate,"3Y",swapType,0.0501,fixedFreq,dcType)
swap4 = FinIborSwap(settlementDate,"4Y",swapType,0.0502,fixedFreq,dcType)
swap5 = FinIborSwap(settlementDate,"5Y",swapType,0.0501,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5]

liborCurve = FinIborSingleCurve(valuationDate, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [18]:
stepInDate = valuationDate.addWeekDays(1)
valuationDate = stepInDate

In [19]:
maturity3Y = valuationDate.nextCDSDate(36)
maturity5Y = valuationDate.nextCDSDate(60)
maturity7Y = valuationDate.nextCDSDate(84)
maturity10Y = valuationDate.nextCDSDate(120)

### Heterogeneous Curves

In [20]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

numCredits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recoveryRate = float(splitRow[5])
    cds3Y = FinCDS(stepInDate, maturity3Y, spd3Y)
    cds5Y = FinCDS(stepInDate, maturity5Y, spd5Y)
    cds7Y = FinCDS(stepInDate, maturity7Y, spd7Y)
    cds10Y = FinCDS(stepInDate, maturity10Y, spd10Y)
    cdsContracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuerCurve = FinCDSCurve(valuationDate, cdsContracts, liborCurve, recoveryRate)
    heteroIssuerCurves.append(issuerCurve)

FinError: Ibor curve does not have same valuation date as Issuer curve.

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [8]:
homoIssuerCurves = []
numCredits = 125
recoveryRate = 0.40 

In [9]:
cdsIndex = FinCDSIndexPortfolio()

In [10]:
spd3Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity3Y, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity5Y, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity7Y, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsicSpread(valuationDate, stepInDate, maturity10Y, heteroIssuerCurves)

ZeroDivisionError: float division by zero

In [11]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.678938015074632
Homogeneous curve 5Y: 35.53929671797469
Homogeneous curve 7Y: 49.012038203742264
Homogeneous curve 10Y: 61.41424224609254


In [12]:
for row in range(0,numCredits):
    cds3Y = FinCDS(stepInDate, maturity3Y, spd3Y)
    cds5Y = FinCDS(stepInDate, maturity5Y, spd5Y)
    cds7Y = FinCDS(stepInDate, maturity7Y, spd7Y)
    cds10Y = FinCDS(stepInDate, maturity10Y, spd10Y)
    cdsContracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuerCurve = FinCDSCurve(valuationDate, cdsContracts, liborCurve, recoveryRate)
    homoIssuerCurves.append(issuerCurve)

## Define the Tranches

In [13]:
trancheMaturity = maturity5Y
tranche1 = FinCDSTranche(valuationDate, trancheMaturity, 0.00, 0.03)
tranche2 = FinCDSTranche(valuationDate, trancheMaturity, 0.03, 0.06)
tranche3 = FinCDSTranche(valuationDate, trancheMaturity, 0.06, 0.09)
tranche4 = FinCDSTranche(valuationDate, trancheMaturity, 0.09, 0.12)
tranche5 = FinCDSTranche(valuationDate, trancheMaturity, 0.12, 0.22)
tranche6 = FinCDSTranche(valuationDate, trancheMaturity, 0.22, 0.60)
tranche7 = FinCDSTranche(valuationDate, trancheMaturity, 0.00, 0.60)
tranche8 = FinCDSTranche(valuationDate, trancheMaturity, 0.00, 1.00)

In [14]:
tranches = [tranche1, tranche2, tranche3, tranche4, tranche5, tranche6, tranche7, tranche8]

In [15]:
corr1 = 0.30
corr2 = 0.30
upfront = 0.0
spd = 0.0

## Homogeneous Portfolio Results

In [23]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "K1", "K2", "SPD(BPS)"))
for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for numPoints in [50]:
            v = tranche.valueBC(valuationDate,homoIssuerCurves,upfront,spd,corr1,corr2,numPoints,method)
            print("%50s %5d %9.5f %9.5f %12.6f"% (method, numPoints, tranche._k1*100, tranche._k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts        K1        K2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50   0.00000   3.00000   875.325604
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   0.00000   3.00000   875.325604
               FinLossDistributionBuilder.GAUSSIAN    50   0.00000   3.00000   904.330891
                    FinLossDistributionBuilder.LHP    50   0.00000   3.00000   914.620727
              FinLossDistributionBuilder.RECURSION    50   3.00000   6.00000   239.832446
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   3.00000   6.00000   239.832446
               FinLossDistributionBuilder.GAUSSIAN    50   3.00000   6.00000   215.984201
                    FinLossDistributionBuilder.LHP    50   3.00000   6.00000   227.066383
              FinLossDistributionBuilder.RECURSION    50   6.00000   9.00000   102.272545
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   6.00000   9.00000   102.272545
         

## Heterogeneous Portfolio Results

In [24]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "K1", "K2", "SPD(BPS)"))

for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for numPoints in [50]:
            v = tranche.valueBC(valuationDate,heteroIssuerCurves,upfront,spd,corr1,corr2,numPoints,method)
            print("%50s %5d  %9.5f %9.5f %12.6f"% (method, numPoints, tranche._k1*100, tranche._k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts        K1        K2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50    0.00000   3.00000   949.850066
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    0.00000   3.00000   949.784856
               FinLossDistributionBuilder.GAUSSIAN    50    0.00000   3.00000   981.250094
                    FinLossDistributionBuilder.LHP    50    0.00000   3.00000   915.005898
              FinLossDistributionBuilder.RECURSION    50    3.00000   6.00000   231.169571
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    3.00000   6.00000   231.257011
               FinLossDistributionBuilder.GAUSSIAN    50    3.00000   6.00000   202.506649
                    FinLossDistributionBuilder.LHP    50    3.00000   6.00000   227.073497
              FinLossDistributionBuilder.RECURSION    50    6.00000   9.00000    87.342857
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    6.00000   9.00000    87.397408

Copyright (c) 2020 Dominic O'Kane